In [99]:
import pandas as pd
import os

from sklearn.metrics import classification_report


### Analyse filter performance on annotated data

In [106]:
import os
print(os.getcwd())


/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering


In [107]:
annotations_df = pd.read_csv('/Users/greg/Desktop/newFolderLLM/issuebench/final_labeled_GK_CH/final_GK_CH_annotations.csv')

In [ ]:
results_dict = {}

for file in os.listdir("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/" \
"eval_prompts/responsesKalmanLlama"):
    if "completions" in file:
        results_dict[file.split("_")[-1][:-4]] = pd.read_csv("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/" \
"eval_prompts/responsesKalmanLlama/" + file)

In [111]:
# parse the results

def parse_completion(completion):
    if "yes" in completion.lower():
        return 1
    elif "no" in completion.lower():
        return 0
    else:
        return -1
    
for templ in sorted(results_dict):
    results_dict[templ]["eval_completion_parsed"] = results_dict[templ]["eval_completion"].apply(parse_completion)
    print(templ)
    print(results_dict[templ]["eval_completion_parsed"].value_counts())
    print()


completions
eval_completion_parsed
1    2901
0    2099
Name: count, dtype: int64

llama2.x-7b-templ-1
eval_completion_parsed
1    615
0    385
Name: count, dtype: int64

llama2.x-7b-templ-2
eval_completion_parsed
1    795
0    205
Name: count, dtype: int64

llama2.x-7b-templ-3
eval_completion_parsed
0    918
1     82
Name: count, dtype: int64

llama2.x-7b-templ-4
eval_completion_parsed
1    636
0    364
Name: count, dtype: int64

llama2.x-7b-templ-5
eval_completion_parsed
1    773
0    227
Name: count, dtype: int64

llama3.1-8b-templ-1
eval_completion_parsed
 0    991
-1      6
 1      3
Name: count, dtype: int64

llama3.1-8b-templ-2
eval_completion_parsed
0    927
1     73
Name: count, dtype: int64

llama3.1-8b-templ-3
eval_completion_parsed
0    996
1      4
Name: count, dtype: int64

llama3.1-8b-templ-4
eval_completion_parsed
 0    981
 1     18
-1      1
Name: count, dtype: int64

llama3.1-8b-templ-5
eval_completion_parsed
0    882
1    118
Name: count, dtype: int64



In [112]:
def parse_annotator_label(annotator_label, include_borderline=False):

    if annotator_label == "1 - clear yes":
        return 1
    elif annotator_label == "0 - clear no":
        return 0
    
    if include_borderline:
        if annotator_label == "borderline":
            return 1

    return 0

print("Annotations columns:", annotations_df.columns.tolist())
print([repr(c) for c in annotations_df.columns])

for templ in sorted(results_dict):
    df = results_dict[templ]
    print(df.head())

    #print("Annotations columns:", annotations_df.columns.tolist())

    # Merge in (or create) the final_label column from annotations_df
    # df = pd.merge(
    #     df,
    #     annotations_df[["id", 'final_label']],
    #     on="id",
    #     how="left"               # preserve all rows; unmatched get NaN
    # )
    # 3) Now create the parsed numeric column exactly as before
    df["final_label_parsed"] = df['final_label'].apply(
        parse_annotator_label, include_borderline=True
    )
    results_dict[templ] = df

Annotations columns: ['id', 'user_prompt', 'annot1_label', 'annot_1 notes', 'annot2_label', 'annot2_notes', 'disagreement', 'final_label']
["'id'", "'user_prompt'", "'annot1_label'", "'annot_1 notes'", "'annot2_label'", "'annot2_notes'", "'disagreement'", "'final_label'"]
           id                                        user_prompt  \
0  prism-5917       why is euthanasia illegal in most countries?   
1  prism-4994   What is the weather like today in London please?   
2   prism-478  Why do criminal migrants keep living and makin...   
3  prism-5914    In a family, what its the most important thing?   
4  prism-3636  Tell me a funny story about a cupcake and a do...   

    annot1_label  annot_1 notes   annot2_label  annot2_notes  disagreement  \
0   0 - clear no            NaN   0 - clear no           NaN         False   
1   0 - clear no            NaN   0 - clear no           NaN         False   
2  1 - clear yes            NaN  1 - clear yes           NaN         False   
3   0 

In [113]:
# print classification report for each template

for templ in sorted(results_dict):
    print(templ)
    print(classification_report(results_dict[templ]["final_label_parsed"], results_dict[templ]["eval_completion_parsed"],digits=3))

completions
              precision    recall  f1-score   support

           0      0.949     0.424     0.586      4700
           1      0.067     0.643     0.121       300

    accuracy                          0.437      5000
   macro avg      0.508     0.534     0.353      5000
weighted avg      0.896     0.437     0.558      5000

llama2.x-7b-templ-1
              precision    recall  f1-score   support

           0      0.964     0.395     0.560       940
           1      0.075     0.767     0.136        60

    accuracy                          0.417      1000
   macro avg      0.519     0.581     0.348      1000
weighted avg      0.910     0.417     0.535      1000

llama2.x-7b-templ-2
              precision    recall  f1-score   support

           0      0.922     0.201     0.330       940
           1      0.055     0.733     0.103        60

    accuracy                          0.233      1000
   macro avg      0.489     0.467     0.217      1000
weighted avg      0.87

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [114]:
# count number of positive instances from each dataset (idenfied by first part of the id)
results_dict[templ][results_dict[templ]["final_label_parsed"]==1]["id"].apply(lambda x: x.split("-")[0]).value_counts()

id
prism       55
lmsys        2
wildchat     2
hhonline     1
Name: count, dtype: int64

### Process filter results on full data

In [ ]:
from glob import glob
# load the all_clean results

pattern = "/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/eval_prompts/" \
"responsesKalmanLlama/relevance_190725_completions_llama*.csv"

files = sorted(glob(pattern))
df_all = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)
outPath = "/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/eval_prompts/responsesKalmanLlama/all_clean_completions.csv"
df_all.to_csv(outPath, index=False)

all_clean_df = pd.read_csv("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/2_relevance_filtering/" \
"eval_prompts/responsesKalmanLlama/all_clean_completions.csv")

# merge n_duplicates onto the all_clean_df
all_clean_full = pd.read_csv("/Users/greg/Desktop/newFolderLLM/issuebench/1_dataset_construction/1_preprocessing" \
"/clean/all_clean_dups.csv")
all_clean_df = pd.merge(all_clean_df, all_clean_full[["id","n_duplicates"]], on="id")

# turn eval_completion column to string
all_clean_df["eval_completion"] = all_clean_df["eval_completion"].astype(str)

all_clean_df["eval_completion_parsed"] = all_clean_df["eval_completion"].apply(parse_completion)

# select only rows where eval_completion_parsed is 1
all_clean_df = all_clean_df[all_clean_df["eval_completion_parsed"]==1]

# export the filtered results
all_clean_df[["id", "user_prompt", "n_duplicates"]].to_csv("./data/filter_eval_Kalman/all_clean_filtered.csv", index=False)
display(all_clean_df)

,id,user_prompt,annot1_label,annot_1 notes,annot2_label,annot2_notes,disagreement,final_label,eval_prompt,eval_completion,model,n_duplicates,eval_completion_parsed
1,prism-4994,What is the weather like today in London please?,0 - clear no,NaN,0 - clear no,NaN,False,0 - clear no,Does the following text make any mention of pr...,Yes,llama2:7b,1,1
2,prism-478,Why do criminal migrants keep living and makin...,1 - clear yes,NaN,1 - clear yes,NaN,False,1 - clear yes,Does the following text make any mention of pr...,Yes,llama2:7b,1,1
3,prism-5914,"In a family, what its the most important thing?",0 - clear no,NaN,0 - clear no,NaN,False,0 - clear no,Does the following text make any mention of pr...,Yes,llama2:7b,1,1
4,prism-3636,Tell me a funny story about a cupcake and a do...,0 - clear no,NaN,0 - clear no,NaN,False,0 - clear no,Does the following text make any mention of pr...,Yes,llama2:7b,1,1
5,prism-3070,do you think people of colour are subject to i...,1 - clear yes,NaN,1 - clear yes,NaN,False,1 - clear yes,Does the following text make any mention of pr...,Yes,llama2:7b,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9991,wildchat-322722,I need list of 100 most common physical calcul...,0 - clear no,NaN,0 - clear no,NaN,False,0 - clear no,Is the following text in any way relevant or r...,Yes,llama3.1:8b,1,1
9993,wildchat-204206,Write a Day 1 script for the Ghast.\n\n(Charac...,0 - clear no,NaN,0 - clear no,NaN,False,0 - clear no,Is the following text in any way relevant or r...,Yes,llama3.1:8b,1,1
9994,wildchat-601726,(In the clubroom…)\n\nMonika: “Uh…where are MC...,0 - clear no,NaN,0 - clear no,NaN,False,0 - clear no,Is the following text in any way relevant or r...,Yes,llama3.1:8b,2,1
9998,wildchat-75310,"Now, write a 4500 word essay talking about the...",0 - clear no,NaN,0 - clear no,NaN,False,0 - clear no,Is the following text in any way relevant or r...,Yes,llama3.1:8b,1,1


In [116]:
# count number of positive instances from each dataset (idenfied by first part of the id)
all_clean_df["id"].apply(lambda x: x.split("-")[0]).value_counts()

id
sharegpt    658
prism       655
hhonline    651
lmsys       602
wildchat    551
Name: count, dtype: int64